In [6]:
import os
from PIL import Image
import torch
import torchvision.transforms as transforms
import timm

# Define the TIMM model name
model_name = 'efficientnet_b0'

# Load the pre-trained TIMM model
model = timm.create_model(model_name, pretrained=True)
model.eval()

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Specify the folder path containing fruit images
folder_path = './test-multiple_fruits'


rows = []

# Iterate over each file in the folder
for idx, filename in enumerate(os.listdir(folder_path)):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path)
        input_tensor = transform(image).unsqueeze(0)

        with torch.no_grad():
            output = model.forward_features(input_tensor)

        # Flatten the output embedding and resize it to have a length of 2048
        embedding = output.squeeze().view(-1)[:2048].numpy()

        # Create a dictionary for the current image
        image_dict = {
            'id': idx,
            'name': filename,
            'vector': embedding.tolist()
        }

        rows.append(image_dict)

output_dict = {'rows': rows}

In [8]:
import json

# Specify the output file path
output_file = 'output.json'

# Write the output dictionary to the JSON file
with open(output_file, 'w') as f:
    json.dump(output_dict, f)

In [11]:
from secretkey import uri,token
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType, utility,MilvusClient


In [12]:
client = MilvusClient(
    uri=uri,
    token=token
    )

In [18]:
import json
with open("./output.json") as f:
  data = json.load(f)

res = client.search(
    collection_name="fruits",
      data=[data["rows"][25]["vector"]],
    output_fields=["name"]
)

print(res)

{'id': 28, 'name': 'cherries6.jpg', 'vector': [-0.1403714120388031, -0.10077106952667236, -0.07507731020450592, -0.057427484542131424, -0.04276832193136215, -0.07780203223228455, -0.11763405799865723, -0.14375239610671997, -0.07557869702577591, -0.0419229157269001, -0.01606798730790615, -0.021383050829172134, -0.05094166845083237, -0.13162636756896973, -0.06239738687872887, -0.030594106763601303, -0.01931045390665531, -0.0009513640543445945, -0.007757105864584446, -0.04997874051332474, -0.22441865503787994, -0.07831928133964539, -0.03721660375595093, -0.01224578358232975, -0.004706535954028368, -0.009695089422166348, -0.1084664911031723, -0.24675330519676208, -0.1314023733139038, -0.19238059222698212, -0.16872330009937286, -0.14741520583629608, -0.2634614109992981, -0.15524335205554962, -0.23874709010124207, -0.20332011580467224, -0.24891971051692963, -0.27837905287742615, -0.2238645702600479, -0.23420432209968567, -0.1923341006040573, -0.24458618462085724, -0.27222558856010437, -0.273